In [ ]:
%matplotlib widget
import os
import h5py
import json
import glob
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from SiPMStudio.analysis.dark import current_waveforms
from SiPMStudio.analysis.dark import integrate_current

from scipy.signal import find_peaks
from scipy.optimize import curve_fit

In [ ]:
data_path = os.path.join("/Volumes", "TOSHIBA_EXT", "sipm_data_2", "ketek_box", "ketek_ln_12162021", "t2")
json_path = os.path.join("/Volumes", "TOSHIBA_EXT", "sipm_data_2", "ketek_box", "ketek_ln_12162021", "ketek_ln_reprocs_norm.json")
save_path = os.path.join("/Volumes", "TOSHIBA_EXT", "sipm_data_2", "ketek_box", "ketek_ln_12162021", "plots", "pe_spectrums")

if not os.path.exists(save_path):
    os.makedirs(save_path)

In [ ]:
file_list = glob.glob(f"{data_path}/*.h5")
file_list

In [ ]:
file_num = 5

In [ ]:
def gaussian(x, A, mu, sigma):
    return A*np.exp(-(x - mu)**2/(2*sigma**2))

def guess_peaks(n, bins, min_height, min_dist):
    bin_width = bins[1] - bins[0]
    min_bin_dist = min_dist / bin_width
    peaks, amplitudes = find_peaks(n, height=min_height, distance=min_bin_dist)
    return peaks, bins[peaks], amplitudes["peak_heights"]

def fit_peaks(n, bins, peaks, peak_locs, amplitudes, fit_width=15):
    gauss_params = []
    gauss_errors = []
    bin_centers = (bins[1:] + bins[:-1]) / 2
    sigma_guess = (peak_locs[1] - peak_locs[0])
    for i, peak in enumerate(peaks):
        left = peak-fit_width
        right = peak+fit_width
        if left < 0:
            left = 0
        coeffs, covs = curve_fit(gaussian, bin_centers[left: right], n[left: right], p0=[amplitudes[i], peak_locs[i], sigma_guess])
        gauss_params.append(coeffs)
        gauss_errors.append(np.sqrt(np.diag(covs)))
    return gauss_params, gauss_errors

def write_to_json(json_name, h5_file_name, gauss_params, gauss_errors):
    head, tail = os.path.split(h5_file_name)
    reproc_dict = None
    with open(json_name, "r") as json_file:
        reproc_dict = json.load(json_file)
        
    gauss_params_np = np.array(gauss_params).T
    gauss_errors_np = np.array(gauss_errors).T
    
    reproc_dict["processes"]["normalize_charge"][tail]["peak_locs"] = list(gauss_params_np[1])
    reproc_dict["processes"]["normalize_charge"][tail]["peak_errors"] = list(gauss_errors_np[1])
    
    with open(json_name, "w") as json_file:
        json.dump(reproc_dict, json_file, indent=4)

In [ ]:
test_file = h5py.File(file_list[file_num], "r+")

In [ ]:
time = np.arange(0, test_file["/processed/channels/sipm/blr_wf"].shape[1])*test_file["dt"][()]
waveforms = test_file["/processed/channels/sipm/blr_wf"][:]
adc_v = test_file["/processed/channels/sipm/adc_to_v"][()]

In [ ]:
plt.figure()
plt.plot(time, waveforms[100]*adc_v*1e3)
plt.xlabel("Time (ns)")
plt.ylabel("Amplitude (mV)")

In [ ]:
charges = test_file["/processed/channels/sipm/charge"][:]
charges.shape

In [ ]:
plt.figure()
n, bins, patches = plt.hist(charges, bins=1000, range=[-0.2e-12, 5e-12], histtype="step", color=sns.color_palette()[0])
n, bins, patches = plt.hist(charges, bins=1000, range=[-0.2e-12, 5e-12], histtype="stepfilled", color=sns.color_palette()[0], alpha=0.15)
plt.xlabel("Integrated Charge (C)")
plt.ylabel("Counts")
plt.yscale("log")
bias = test_file["bias"][()]
plt.savefig(os.path.join(save_path, f"pe_spec_{bias}.png"), dpi=600)

In [ ]:
peaks, peak_locs, amplitudes = guess_peaks(n, bins, 14, 2.5e-13)

In [ ]:
plt.figure()
n, bins, patches = plt.hist(charges, bins=1000, range=[-0.2e-12, 5e-12], histtype="step", color=sns.color_palette()[0])
n, bins, patches = plt.hist(charges, bins=1000, range=[-0.2e-12, 5e-12], histtype="stepfilled", color=sns.color_palette()[0], alpha=0.15)
plt.scatter(peak_locs, amplitudes)
plt.xlabel("Integrated Charge (C)")
plt.ylabel("Counts")
plt.yscale("log")

In [ ]:
gauss_params, gauss_errors = fit_peaks(n, bins, peaks, peak_locs, amplitudes, fit_width=15)

In [ ]:
x = np.linspace(-0.2e-12, 4e-12, 1000)

In [ ]:
plt.figure()
for i, params in enumerate(gauss_params):
    plt.plot(x, gaussian(x, *params), color=sns.color_palette()[i])
n, bins, patches = plt.hist(charges, bins=1000, range=[-0.2e-12, 5e-12], histtype="step", color=sns.color_palette()[0])
n, bins, patches = plt.hist(charges, bins=1000, range=[-0.2e-12, 5e-12], histtype="stepfilled", color=sns.color_palette()[0], alpha=0.15)
plt.xlabel("Integrated Charge (C)")
plt.ylabel("Counts")
plt.xlim(-0.2e-12, 5e-12)
plt.ylim(1, 10000)
plt.yscale("log")
plt.savefig(os.path.join(save_path, f"pe_spec_{bias}_fit.png"), dpi=600)

In [ ]:
#gauss_params.insert(0, np.zeros(3))
#gauss_errors.insert(0, np.ones(3)*1e-15)

In [ ]:
write_to_json(json_path, file_list[file_num], gauss_params, gauss_errors)

In [ ]:
len(test_file["bias"].shape)

In [ ]:
test_file["processed"].keys()

In [ ]:
test_file.close()